In [1]:
import numpy as np
import json

# Расчет дерева

In [ ]:
edge_list = {'Заказ такси': ['Заказать', 'Не заказать'],
             'Заказать': ['Заказали и дождь', 'Заказали и нет дождя'],
             'Не заказать': ['Не заказали и дождь', 'Не заказали и нет дождя'],
             'Заказали и дождь': ['ЗДЕ', 'ЗДН'],
             'Заказали и нет дождя': ['ЗНЕ', 'ЗНН'],
             'ЗДЕ': ['ЗДЕЗ', 'ЗДЕН'],
             'ЗДН': ['ЗДНЗ', 'ЗДНН'],
             'ЗНЕ': ['ЗНЕЗ', 'ЗНЕН'],
             'ЗНН': ['ЗННЗ', 'ЗННН']}
vert_props = {'Заказ такси': ('rdec', -1, 1),
              'Заказать': ('prob', 0, 1),
              'Не заказать': ('term', 1000, 1),
              'Заказали и дождь': ('dec', -1, 0.6),
              'Заказали и нет дождя': ('dec', -1, 0.4),
              'ЗДЕ': ('prob', 0, 1),
              'ЗДН': ('prob', 0, 1),
              'ЗНЕ': ('prob', 0, 1),
              'ЗНН': ('prob', 0, 1),
              'ЗДЕЗ': ('term', 2500, 0.1),
              'ЗДЕН': ('term', 500, 0.9),
              'ЗДНЗ': ('term', 2150, 0.3),
              'ЗДНН': ('term', 150, 0.7),
              'ЗНЕЗ': ('term', 2500, 0.05),
              'ЗНЕН': ('term', 500, 0.95),
              'ЗННЗ': ('term', 2150, 0.1),
              'ЗННН': ('term', 150, 0.9),
              }

In [13]:
edge_list = {'Строить ЖК?': ['Строить', 'Не строить'],
             'Строить': ['Рекламная кампания удачная', 'Рекламная кампания не удачная'],
             #'Не строить': ['0'],
             'Рекламная кампания удачная': ['Не строить с рекл.', 'Большой ЖК', 'Маленький ЖК'],
             'Рекламная кампания не удачная': ['Не строить c рекл.'],
             'Большой ЖК': ['Спрос снизится для Б', 'Спрос не снизится для Б'],
             'Маленький ЖК': ['Спрос снизится для М', 'Спрос не снизится для М']
             }
vert_props = {'Строить ЖК?': ('rdec', 1, 1),
              'Строить': ('prob', 0, 1),
              'Не строить': ('term', 0, 1),
              'Рекламная кампания удачная': ('dec', 1, 0.25),
              'Рекламная кампания не удачная': ('term', -500000, 0.75),
              'Большой ЖК': ('prob', 0, 1),
              'Маленький ЖК': ('prob', 0, 1),
              'Не строить с рекл.': ('term', -500000, 1),
              'Спрос снизится для Б': ('term', -115000000, 0.4),
              'Спрос не снизится для Б': ('term', 20000000, 0.6),
              'Спрос снизится для М': ('term', -10000000, 0.4),
              'Спрос не снизится для М': ('term', 65000000, 0.6),
              }

In [22]:
with open('data_tree.json', 'w', encoding='utf-8') as f:
    json.dump([edge_list, vert_props], f, ensure_ascii=False, indent=4)
  

In [25]:
with open('data_tree.json', 'r', encoding='utf-8') as f:
  a = json.load(f)
  

In [15]:
vert_types = ['dec', 'prob', 'term', 'rprob', 'rdec']

In [16]:
def tree_dfs(v, ancestor):
  if vert_props[v][0] == 'term':
    try :
      rec_stack[ancestor].append(float(vert_props[v][1] * vert_props[v][2]))
    except:
      rec_stack[ancestor] = [float(vert_props[v][1] * vert_props[v][2])]
    return
  for suc in edge_list[v]:
    tree_dfs(suc, v)

  if vert_props[v][0] == 'prob':
      try :
        rec_stack[ancestor].append(sum(rec_stack[v])* vert_props[v][2])
      except:
        rec_stack[ancestor] = [sum(rec_stack[v])* vert_props[v][2]]
    
  if vert_props[v][0] == 'dec':
    if vert_props[v][1] == 1:
      try :
        rec_stack[ancestor].append(max(rec_stack[v])* vert_props[v][2])
      except:
        rec_stack[ancestor] = [max(rec_stack[v])* vert_props[v][2]]
    if vert_props[v][1] == -1:
        try :
          rec_stack[ancestor].append(min(rec_stack[v])* vert_props[v][2])
        except:
          rec_stack[ancestor] = [min(rec_stack[v])* vert_props[v][2]]
    #print(rec_stack, v)
  
  if vert_props[v][0] == 'rprob':
    print(sum(rec_stack[v]))
  if vert_props[v][0] == 'rdec':
    if vert_props[v][1] == 1:
      print(max(rec_stack[v]))
    else:
      print(min(rec_stack[v]))

In [17]:
rec_stack = dict()
tree_dfs('Строить ЖК?', 'Строить ЖК?')

8375000.0


In [33]:
rec_stack

{'Большой ЖК': [-46000000.0, 12000000.0],
 'Маленький ЖК': [-4000000.0, 39000000.0],
 'Рекламная кампания удачная': [-500000.0, -34000000.0, 35000000.0],
 'Строить': [8750000.0, -375000.0],
 'Строить ЖК?': [8375000.0, 0.0]}

In [18]:
def get_dec(v):
  if vert_props[v][0] not in ['dec', 'rprob', 'rdec']:
    print('Error! Choose decision vertex')
  else:
    if vert_props[v][1] == 1:
      print('BEST for '+v+': ' + edge_list[v][np.argmax(rec_stack[v])])
      print(max(rec_stack[v]))
    else:
      print('BEST for '+v+': '+ edge_list[v][np.argmin(rec_stack[v])])
      print(min(rec_stack[v]))



In [19]:
get_dec('Строить ЖК?')

BEST for Строить ЖК?: Строить
8375000.0


In [27]:
get_dec('Рекламная кампания удачная')

BEST for Рекламная кампания удачная: Маленький ЖК
35000000.0


# Отрисовка

In [28]:
import graphviz

In [31]:
dot = graphviz.Digraph('Tree', comment='The Round Table', engine='sfdp')

for k in vert_props.keys():
  if vert_props[k][0] == 'dec':
    dot.node(k, k, shape='square', color='lightgrey', style='filled')

  if vert_props[k][0] == 'term':
    dot.node(k, k, shape='egg')

  if vert_props[k][0] == 'prob':
    dot.node(k, k, shape='triangle', color='lightblue', style='filled')

  if vert_props[k][0] == 'rprob':
    dot.node(k, k, shape='Mtriangle', color='red', style='filled')

  if vert_props[k][0] == 'rdec':
    dot.node(k, k, shape='Msquare', color='red', style='filled')

for v in edge_list.keys():
  for vv in edge_list[v]:
    dot.edge(v, vv, constraint='false')
dot.render('tree')

'tree.pdf'

In [30]:
dot.save('tree_dot', )

'tree_dot'